In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [4]:
embed = hub.Module(module_url)

In [6]:
read = pd.read_csv('dataset/test_dataset', delimiter=',', header=None)
data_articles = read.iloc[:,:].values

In [7]:
read = pd.read_csv('dataset/cleanDataTables', delimiter=',', header=None)
data_tables = read.iloc[:,:].values

## creating the articles embedings

In [9]:
articles_title = []
articles_id = []

for article_id, title, text in tqdm(data_articles):
    
    articles_id.append(article_id)
    
    articles_title.append(title)

100%|██████████| 6600/6600 [00:00<00:00, 109310.34it/s]


In [10]:
with tf.Session() as sess:
    
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            
    embedding_articles = sess.run(embed(articles_title))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
article_dense_vector = []

for current_embedding in embedding_articles:
    
    article_dense_vector.append(current_embedding)

In [12]:
len(article_dense_vector)

6600

## creating the tables embeddings

In [14]:
tables_title = []

for current_table in tqdm(data_tables):
    
    tables_title.append(str(current_table[1]))

100%|██████████| 298793/298793 [00:00<00:00, 356276.52it/s]


In [15]:
with tf.Session() as sess:
    
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            
    embedding_tables = sess.run(embed(tables_title))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [16]:
tables_dense_vector = []

for current_embedding in embedding_tables:
    
    tables_dense_vector.append(current_embedding)

In [17]:
len(tables_dense_vector)

298793

## methods for getting the ranked tables and save the accuracy

In [18]:
def get_id_ranked_tables(top_k,distance_vector):

    id_ranked_tables = []

    for current_top_k in top_k:
        
        index = np.where(distance_vector == current_top_k)
         
        index_colummun = index[0][0]
        
        id_ranked_tables.append(data_tables[index_colummun][0])

    return id_ranked_tables

In [19]:
def get_accuracy(id_ranked_tables, id_query_goal):

    accuracy = 0

    for id_table in id_ranked_tables:
    
        if id_table == id_query_goal:
    
            accuracy = 1
            
            break;

    return accuracy

In [20]:
def save_accuracy(k,accuracy):
    
    if k == 1:
            
        average_top1.append(accuracy)
        
    if k == 10:
            
        average_top10.append(accuracy)
        
    if k == 100:
            
        average_top100.append(accuracy)
        
    if k == 1000:
            
        average_top1000.append(accuracy)

## computing the cosine similarity

In [ ]:
average_top1 = []
average_top10 = []
average_top100 = []
average_top1000 = []

top_k = [1,10,100,1000]

for i in tqdm(range(len(article_dense_vector))):
    
    distance_vector = pairwise_distances(article_dense_vector[i].reshape(1,512), tables_dense_vector, metric='cosine')
    
    id_query_goal = int(articles_id[i])
    
    for accuracy_k in top_k:
        
        count_top_tables = accuracy_k
        
        top_k_rank = nsmallest(count_top_tables, distance_vector[0])
    
        id_ranked_tables = get_id_ranked_tables(top_k_rank,distance_vector[0])
        
        accuracy_value = get_accuracy(id_ranked_tables,id_query_goal)
        
        #save the accuracy on the list
        save_accuracy(accuracy_k,accuracy_value)

 39%|███▊      | 2557/6600 [2:13:37<3:32:08,  3.15s/it]

In [ ]:
print(str(round(np.mean(average_top1),4))+" (±) "+str(round(np.std(average_top1),4)))
print(str(round(np.mean(average_top10),4))+" (±) "+str(round(np.std(average_top10),4)))
print(str(round(np.mean(average_top100),4))+" (±) "+str(round(np.std(average_top100),4)))
print(str(round(np.mean(average_top1000),4))+" (±) "+str(round(np.std(average_top1000),4)))